# **Import Libraries**

In [9]:
import pandas as pd
import pickle
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup


In [10]:
def get_product_urls(url_main, total_pages):
    """
    Mengambil URL produk dari kategori dengan jumlah halaman yang ditentukan secara manual menggunakan Selenium.
    """
    
    product_urls = []
    base_url = url_main.split('?')[0]  # Ambil base URL tanpa parameter

    options = webdriver.ChromeOptions()
    options.add_argument("enable-automation")
    options.add_argument("useAutomationExtension")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-extensions")
    options.add_argument("--dns-prefetch-disable")
    options.add_argument("window-size=1920,1080")


    driver = webdriver.Chrome(options=options)

    try:
        for page in range(1, total_pages + 1):
            url_with_page = f"{base_url}?ob=5&rt=1,2,3&page={page}"
            print(f"Processing page {page}/{total_pages}: {url_with_page}")
            
            # Akses URL menggunakan WebDriver
            driver.get(url_with_page)
            sleep(3)

            for _ in range(20):
                driver.execute_script("window.scrollBy(0, 250)")
                sleep(1)

            # Ambil halaman HTML setelah scrolling selesai
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # Cari elemen produk berdasarkan class
            product_divs = soup.find_all('div', {"class": "css-bk6tzz e1nlzfl2"})
            
            if not product_divs:
                print(f"  No products found on page {page}. Stopping scraping for this category.")
                break

            for div in product_divs:
                a_tag = div.find('a')
                if a_tag and a_tag.get('href'):
                    href = a_tag['href']
                    if "https://ta.tokopedia.com/promo/v1/clicks/" not in href:
                        product_urls.append(href)
                        #print(f"URL added: {href}")
                    #else:
                        #print(f"Skipping promo URL: {href}")
    finally:
        driver.quit()

    print(f"Total product URLs collected: {len(product_urls)}")
    return product_urls

In [11]:
urlss = ['https://www.tokopedia.com/p/fashion-pria/sepatu-pria?page=1&rt=1,2,3&ob=5'] #ganti link sendiri
for url in urlss:
    product_urls = get_product_urls(url,104)#ganti jumlah halaman

Processing page 1/104: https://www.tokopedia.com/p/fashion-pria/sepatu-pria?ob=5&rt=1,2,3&page=1
Processing page 2/104: https://www.tokopedia.com/p/fashion-pria/sepatu-pria?ob=5&rt=1,2,3&page=2
Processing page 3/104: https://www.tokopedia.com/p/fashion-pria/sepatu-pria?ob=5&rt=1,2,3&page=3
Processing page 4/104: https://www.tokopedia.com/p/fashion-pria/sepatu-pria?ob=5&rt=1,2,3&page=4
Processing page 5/104: https://www.tokopedia.com/p/fashion-pria/sepatu-pria?ob=5&rt=1,2,3&page=5
Processing page 6/104: https://www.tokopedia.com/p/fashion-pria/sepatu-pria?ob=5&rt=1,2,3&page=6
Processing page 7/104: https://www.tokopedia.com/p/fashion-pria/sepatu-pria?ob=5&rt=1,2,3&page=7
Processing page 8/104: https://www.tokopedia.com/p/fashion-pria/sepatu-pria?ob=5&rt=1,2,3&page=8
Processing page 9/104: https://www.tokopedia.com/p/fashion-pria/sepatu-pria?ob=5&rt=1,2,3&page=9
Processing page 10/104: https://www.tokopedia.com/p/fashion-pria/sepatu-pria?ob=5&rt=1,2,3&page=10
Processing page 11/104: http

In [12]:
len(product_urls)

3290

In [13]:
with open('product_url_1-104.pickle', 'wb') as file:
    # Serialize and write the variable to the file
    pickle.dump(product_urls, file)

In [7]:
with open('product_url_1-104.pickle', 'rb') as file_1:
  product_urls = pickle.load(file_1)

In [23]:
reviews = []
ratings = []

options = webdriver.ChromeOptions()
options.add_argument("enable-automation")
options.add_argument("useAutomationExtension")
options.add_argument("--no-sandbox")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument("window-size=1920,1080")


driver = webdriver.Chrome(options=options)


try:
    for product_url in range (791, len(product_urls)):
        print(f"Processing: {product_urls[product_url]}")
        driver.get(product_urls[product_url])
        sleep(3)

        while True:
            # Scroll the page to load all reviews
            for _ in range(20):
                driver.execute_script("window.scrollBy(0, 250)")
                sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # Extract reviews and ratings
            for product in soup.find_all('div', {"class": "css-1k41fl7"}):
                review_element = product.find('span', {"data-testid": "lblItemUlasan"})
                reviews.append(review_element.get_text() if review_element else 'None')

                rating_element = product.find('div', {"class": "rating"})
                ratings.append(rating_element.get('aria-label') if rating_element else 'None')

            # Check if "Next" button exists and is enabled
            try:
                next_button_container = driver.find_element(By.CLASS_NAME, "css-1xqkwi8")
                next_button = next_button_container.find_element(
                    By.XPATH, './/button[contains(@class, "css-16uzo3v-unf-pagination-item") and @aria-label="Laman berikutnya"]'
                )
                is_disabled = next_button.get_attribute("disabled")  # Check if button is disabled
                if is_disabled:
                    print(f"Data ke - {product_url}")
                    print("No more pages to navigate for this product.")
                    break

                # Scroll to and click the next button
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                driver.execute_script("arguments[0].click();", next_button)
                sleep(2)
            except (NoSuchElementException, TimeoutException):
                print(f"Data ke - {product_url}")
                print("No 'Next' button found. Moving to next product.")
                break

finally:
    driver.quit()

Processing: https://www.tokopedia.com/fukumuku/sepatu-vans-authentic-triple-white-global-market-sepatu-skateboard-43?extParam=ivf%3Dfalse
Data ke - 791
No 'Next' button found. Moving to next product.
Processing: https://www.tokopedia.com/archive-gotretshop/crocs-classic-all-terrain-camo-sandal-crocs-all-terrain-grey-army-40?extParam=ivf%3Dfalse
Data ke - 792
No 'Next' button found. Moving to next product.
Processing: https://www.tokopedia.com/lilingga/sandal-pria-wanita-slip-on-sandal-slide-premium-minimalis-hitam-38?extParam=ivf%3Dfalse
Data ke - 793
No 'Next' button found. Moving to next product.
Processing: https://www.tokopedia.com/istana11/sandal-pria-pakalolo-original-asli-kulit-sandal-asli-pakalolo-cokelat-39?extParam=ivf%3Dfalse
Data ke - 794
No 'Next' button found. Moving to next product.
Processing: https://www.tokopedia.com/sneakerslegit/vans-sepatu-slipon-checkerboard-original?extParam=ivf%3Dfalse
Data ke - 795
No 'Next' button found. Moving to next product.
Processing: htt

KeyboardInterrupt: 

In [24]:
print(f"Scraped {len(reviews)} reviews and {len(ratings)} ratings.")
data = pd.DataFrame({'Review': reviews, 'Rating': ratings})

Scraped 1449 reviews and 1449 ratings.


In [25]:
data.to_csv("reviews_2_lanjut3.csv", index=False)